## Put data on the input map
After map generation, we have to parse data coming from Bogota´s weather map

In [1]:
import json
import pandas as pd
from shapely.ops import cascaded_union
from pandas.io.json import json_normalize
from shapely.geometry import Point, Polygon
import geopandas as gpd
from folium import Map, Marker, GeoJson, features
from h3 import h3
import requests
import zipfile
import xmltodict, io
import time
import branca.colormap as cm

In [2]:
# Print actual time to insert on API
actual_time = round(time.time())
print(actual_time)

1588897057


In [3]:
# Fetch data at the actual Bogota time.
resp = requests.get("http://imrad.sire.gov.co:8080/radar/reflectividad.kmz?rand=%s" % actual_time)
resp= zipfile.ZipFile(io.BytesIO(resp.content))
resp.extractall('reflec')
resp = resp.open('doc.kml', 'r').read()
resp = xmltodict.parse(resp)
img_radar = resp["kml"]["Folder"]["GroundOverlay"]["Icon"]["href"]
resp

OrderedDict([('kml',
              OrderedDict([('@xmlns', 'http://earth.google.com/kml/2.2'),
                           ('Folder',
                            OrderedDict([('name',
                                          '1399BOG-Intensity (Horizontal)-Mon May 4 15:09:09 2020 GMT'),
                                         ('ExtendedData',
                                          OrderedDict([('@xmlns:edge',
                                                        'http://eecradar.com'),
                                                       ('edge:Moment',
                                                        'Intensity (Horizontal)'),
                                                       ('edge:MomentAbbreviation',
                                                        'ZH')])),
                                         ('lookAt',
                                          OrderedDict([('longitude',
                                                        '-74.11397552490234'),


### Get Lat and long per pixel from Layer

In [4]:
# First, get distance for map´s latitude and longitude
img_loc = resp["kml"]["Folder"]["GroundOverlay"]["LatLonBox"]
hor_loc = float(img_loc["east"]) - float(img_loc["west"])
ver_loc = float(img_loc["north"]) - float(img_loc["south"])
print("Distance horizontal " + str(hor_loc) + " and vertical " + str(ver_loc) + "layer")

Distance horizontal 1.0815526954502985 and vertical 1.085170655991746layer


In [5]:
# Tranform latitud & longitude metrics in pixel sizes
hor_pix = hor_loc/120
ver_pix = ver_loc/120
print("horisontal and vertical per pixel " + str(hor_pix) + ", " + str(ver_pix))

horisontal and vertical per pixel 0.009012939128752487, 0.009043088799931216


In [6]:
# Get data from notebook part 1
with open('source/map_area_hex8.json') as f:
    bogota_hex8 = json.load(f)
bogota_hex = json.loads(bogota_hex8)
bogota_hex =  gpd.GeoDataFrame(bogota_hex['features'])
bogota_hex.head(2)

,type,id,geometry,properties
0,Feature,8866e46669fffff,"{'type': 'Polygon', 'coordinates': [[[-74.2457...",{'value': 0}
1,Feature,8866e43369fffff,"{'type': 'Polygon', 'coordinates': [[[-74.0036...",{'value': 0}


In [7]:
bogota_hex['centroid'] = bogota_hex['id'].map(lambda x: h3.h3_to_geo(x))
bogota_hex.head(2)

,type,id,geometry,properties,centroid
0,Feature,8866e46669fffff,"{'type': 'Polygon', 'coordinates': [[[-74.2457...",{'value': 0},"[4.770617585672407, -74.24528195485323]"
1,Feature,8866e43369fffff,"{'type': 'Polygon', 'coordinates': [[[-74.0036...",{'value': 0},"[5.040572085984343, -74.00316457955307]"


### Map Centroids with Image pixels

In [8]:
# Get pixel x, y coordinates from a geolocation list
def getPixels_xy(_list):
    img_x_axis = _list[1] - float(img_loc["east"])
    img_y_axis = float(img_loc["north"]) - _list[0]
    return [round(img_x_axis/hor_pix), round(img_y_axis/ver_pix)]

In [9]:
# Returns RGB colors from input pixel position 
def pixFunction(_list):
    try: return pix[_list[0],_list[1]]
    except: return (0,0,0,0)

In [10]:
# Write CSV with pixel image on map geolocation
bogota_hex['pixel_coord'] = bogota_hex['centroid'].map(lambda x: getPixels_xy(x))
bogota_hex["coordinates"] = bogota_hex["geometry"].map(lambda x: x["coordinates"])
bogota_hex['rgb'] = bogota_hex['pixel_coord'].map(lambda x: pixFunction(x)[:3])
bogota_hex.head(2)

,type,id,geometry,properties,centroid,pixel_coord,coordinates,rgb
0,Feature,8866e46669fffff,"{'type': 'Polygon', 'coordinates': [[[-74.2457...",{'value': 0},"[4.770617585672407, -74.24528195485323]","[-75, 49]","[[[-74.24579744928519, 4.77546513454456], [-74...","(0, 0, 0)"
1,Feature,8866e43369fffff,"{'type': 'Polygon', 'coordinates': [[[-74.0036...",{'value': 0},"[5.040572085984343, -74.00316457955307]","[-48, 20]","[[[-74.00368257676763, 5.045404510361257], [-7...","(0, 0, 0)"


In [11]:
# Define value levels by researching different RGB colors
def rgbtoRain(_tuple):
    a=0
    if _tuple[0] < 130 and _tuple[1] < 130 and _tuple[2] < 130: a = 0
    if _tuple[0] > 130 and _tuple[1] > 130 and _tuple[2] > 130: a = 0
    if _tuple[0] < 130 and _tuple[1] > 130 and _tuple[2] > 130: a = 1
    if _tuple[0] < 130 and _tuple[1] > 130 and _tuple[2] < 130: a = 2
    if _tuple[0] > 130 and _tuple[1] > 130 and _tuple[2] < 130: a = 3
    if _tuple[0] > 130 and _tuple[1] < 130 and _tuple[2] < 130: a = 4
    return a

In [12]:
def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already, along with the columns hex_id and value '''
    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = {'type': 'Polygon', 'coordinates': row["coordinates"]}  , id=row["id"], properties = {"value" : row["rain_level"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

In [13]:
bogota_hex['rain_level'] = bogota_hex['rgb'].map(lambda x: rgbtoRain(x))
bogota_hex.head(2)

,type,id,geometry,properties,centroid,pixel_coord,coordinates,rgb,rain_level
0,Feature,8866e46669fffff,"{'type': 'Polygon', 'coordinates': [[[-74.2457...",{'value': 0},"[4.770617585672407, -74.24528195485323]","[-75, 49]","[[[-74.24579744928519, 4.77546513454456], [-74...","(0, 0, 0)",0
1,Feature,8866e43369fffff,"{'type': 'Polygon', 'coordinates': [[[-74.0036...",{'value': 0},"[5.040572085984343, -74.00316457955307]","[-48, 20]","[[[-74.00368257676763, 5.045404510361257], [-7...","(0, 0, 0)",0


In [14]:
import branca.colormap as cm
geo = hexagons_dataframe_to_geojson(bogota_hex)

custom_cm = cm.LinearColormap(['white','red', 'blue', 'green'], vmin=0, vmax=4)
    
mapa_bogota = Map(location= [4.626853707360377,-74.0682993486343], zoom_start=11, 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )


GeoJson(
        geo,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': custom_cm(feature['properties']['value']),
            'weight': 1,
            'fillOpacity': 0.1
        }, 
        name = "Hexagons_b" 
    ).add_to(mapa_bogota)

mapa_bogota

NameError: name 'Feature' is not defined